# Ch 5 : Networking

Networking often refers to connecting multiple cimputers together for the purpose of allowing some communication among them. But, for our purposes, we are less interested in allowing computers to communicate with one another and more interested in allowing processes to communicate with one another. Whether the processes are on the same computer or different computers is irrelevant for the techniques that we're going to show.

This chapter will focus on writing Python programs that connect to other processes using the standard socket libarary and then interacting with those other processes.

## Network Clients 

While servers sit and wait for a client to connect to them, clients initiate connections. The Python Standard Library contains implementations of many used networks clients. This section will discuss some of the more common and frequently useful clients.

**socket**

The socket module provides a Python interface to your operating system's socket implementation. This means that you can do whatever can be done to or with sockets, using Python. In case you have never done any network programming before, this chapter does provie a breif overview of networking. It should give you a flavor of what kinds of things you can do with the PYthon networking libraries.

The socket module provides the factory function, socket(). The socket() function, in turn, returns a socket object. While there are a number of arguments to pass to socket() for specifying the kind of socket to create, calling the socket() factory function with no arguments returns a socket object with sensible defaults- a TCP/IP socket:

In [3]:
import socket
s = socket.socket()
s.connect(('192.168.1.15', 80))
s.send("GET / HTTP/1.0\n\n")
s.recv(200)
s.close()

KeyboardInterrupt: 

This example created a socket object called s from the socket() factory function. It then connected to a local default web server, indicated by port 80, which is the default port for HTTP. Then, it sent the server the text string "GET / HTTP/1.0\n\n". Following the send, it received the first 200 bytes of the server's response, which is a 200 ok status message and HTTP headers. Finally, we closed the connection.

The socket methods demonstrated in this example represent the methods that you are likely to find yourself using most often. Connect() establishes a communication channel between your socket object and the remote.

Send() transmits data from your socket object to the remote end. Recv() receives any data that the remote end has sent back. And close() terminates the communication channel between the two sockets. This is a really simple example that shows the ease with which you can create socket object and then send ad receive data over them. 

Now we'll look at a slightly more useful example. Suppose you have a server that is runnning some sort of network application, such as a web server. And suppose that you are interested in watching this server to be sure that, over the course of a day, you can proves that the server itself is still up and that the web server is still listening on some port. 

In [ ]:
import socket
import re
import sys

def check_server(address, port):
    # create a TCP socket
    s = socket.socket()
    print("Attempting to connect to {} on port {}".format(address, port))
    try:
        s.connect((address, port))
        print("Connected to {} on port {}".format(address, port))
        return True
    except socket.error, e:
        print("Connection to {} on port {} failed: {}").format(address, port, e))
        return False
    
if __name__ == "__main__":
    from optparse import OptionParser
    
    parser = OptionParser()
    
    parser.add_option("-a", "--adddress", dest="address", default="localhost",
                     help="ADDRESS for server", metavar="ADDRESS")
    
    parser.add_option("-p", "--port", dest="port", type="int", default=80,
                     help="PORT for server", metavar="PORT")
    
    (options, args) = parser.parse_args()
    print("options: {}, args: {}".format(options, args))
    check = check_server(options.address, options.port)
    print("check_server returned {}".format(check))
    sys.exit(not check)

In [ ]:
port_chechker_tcp.py -a 192.168.1.15 -p 80 && echo "SUCCESS"

- socket based web server checker

In [ ]:
#~/user/bin/env python

import socket
import re
import sys

def check_webserver(address, port, resource):
    # build up HTTP request string
    if not resource.startswith('/'):
        resource = '/' + resource
    request_string = "GET {} HTTP/1.1\r\nHost: {}\r\n\r\n".format(resource, address)
    print('HTTP request:')
    print('|||{}|||'.format(request_string))
    
    # create a TCP socket
    s = socket.socket()
    print("Attempting to connect to {} on port {}".format(address, port))
    try:
        s.connect((address, port))
        print("Connected to {} on port {}".format(address, port))
        s.send(request_string)
        # We should only need the first 100 bytes or so
        rsp = s.recv(100)
        print("Received 100 bytes of HTTP response")
        print("|||{}|||".format(rsp))
    except socket.error, e:
        print("Connection to {} on port {} failed:".format(address, port, e))
        return False
    finally:
        # be a good citizen and close your connection
        print("Closing the connection")
        s.close()
    lines = rsp.splilines()
    print("First line of HTTP response: {}".format(lines[0]))
    try:
        version, status, messgae = re.split(r'\s+', lines[0], 2)
        print("Version: {}, Status: {}, Message: {}".format(version, status, message))
    except ValueError:
        print("Falied to split status line")
        return False
    if status in ['200', '301']:
        print("Success - status was {}".format(status))
        return True
    else:
        print("Status was {}".format(status))
        return False
    

if __name__ == "__main__":
    from optparse import OptionParse
    parser = OptionParser()
    parser.add_option("-a", "--address", dest="address", default="localhost",
                     help="ADDRESS for webserver", metavar="ADDRESS")
    
    parser.add_option("-p", "--port", dest="port", type="int", default=80,
                     help="PORT for webserver", metavar="PORT")
    
    parser.add_option("-r", "--resource", dest="resource", default="index.html",
                     help="RESOURCE to check", metavar="RESOURCE")
    
    (options, args) = parser.parse_args()
    print("options: {}, args: {}".format(options, args))
    check = check_webserver(options.address, options.port, options.resource)
    print("check_webserver returned {}".format(check))
    sys.exit(not check)

This section showed how to construct low-level utilities to connect to network servers and perform basic checks on them. The purpose of these examples was to introduce you to what happens beghind the scenes when clients and servers communicate with one another. If you have an opportunity to write a network component using a higher library than the socket module, you should take it. It is not desirable to spend your time writing network components using a low-level library such as socket.

httplib

In [ ]:
import httplit
import sys

def check_webserver(address, port, resource):
    # create connection
    if not resource.startswith('/'):
        resource = '/' + resource
    try:
        conn = httplib.HTTPConnection(address, port)
        print("HTTP connection created successfully")
        
        # make request
        req = conn.request('GET', resource)
        print("request for {} successfult".format(resource))
        
        # get response
        response = conn.getresponse()
        print("response status: {}".format(response.status))
    except sock.error, e:
        print("HTTP connection failed: {}", e)
        return False
    finally:
        conn.close()
        print("HTTP connection closed successfully")
    if response.status in [200, 301]:
        return True
    else:
        return False

- ftplib

In addition to the socket and httplib modules, the Python Standard Library also contains an FTP client module named ftplib. ftplib is a full-featued FTP client library that will allow you to programmatically perform any tasks you would normally use an FTP client application to perform. For example, you can log in to an FTP server, list files in a particular directory, retrieve files, put files, change directories, and logout, all from within a Python script. You can even use one of the many GUI frameworks available in Python and build your own GUI FTP application.

urllib

## Remote Procedure Call Facilities

Typically, the reason for writing networking code is aht you need interprocess communication. Often, plain IPC, such as HTTP or a plain socket, is good enough. However, there are times when it would be even more useful to execute code in a different process or even on a different computer, as though it were in the same process that the code you are working on is in. If you could, in fact, execute code remotely in some other process from your PYthon program, you might expect that the return values from the remote calls would be Python objects which you could deal more easily wiht than chunks of text through which you have to manually parse. The good news is that there are several tools for remote procedure call functionality.

XML-RPC

## SSH

SSH is an incredibly powerful, widely used protocol. You can also think of it as a tool since the most common implementation includes the same name. SSH allows you to securely connect to remote server, execute shell commands, transfer files, and forward ports in both directions across the connection.

## Twisted

event-driven nework

# Ch 6 : Data

## Using the OS Module to Interact with Data

In [4]:
import os
os.getcwd()

'/Users/kadencho/git/kadensungbincho/Code_followup_book/Python_for_unix_and_linux_system'

In [5]:
os.listdir()

['.txt',
 '20180423.ipynb',
 'study_20180422.ipynb',
 'pysysinfo_func.py',
 'ls_python.py',
 '.ipynb_checkpoints',
 'ch5_20180501.ipynb']

In [6]:
os.stat(".")

os.stat_result(st_mode=16877, st_ino=4139297, st_dev=16777220, st_nlink=9, st_uid=501, st_gid=20, st_size=288, st_atime=1525168955, st_mtime=1525168129, st_ctime=1525168129)

p 200
